In [1]:
import torch
import import_ipynb
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import os

from autoencoder_classes import Encoder, Decoder, ConvAutoencoder

In [2]:
# transforms for data preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'split_dataset')
train_dir = os.path.join(data_dir, 'train')

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

In [3]:
# choose the best device to run on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# new encoder object of the Encoder class defined; dame architecture for the saved weights
encoder = Encoder(in_channels=3) 

# loads saved parameter dictionary from the file into encoder, "map.location" ensures tensors in the file are remapped to the chosen device
encoder.load_state_dict(torch.load("encoder.pth", map_location=device)) 

# moves encoder's parameters and buffers to the selected device
encoder.to(device)

#switches encoder to evaluation mode: turns off training-specific behaviors
encoder.eval()

Encoder(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [4]:
# extract features
train_features = []
train_labels = []

with torch.no_grad():
    for data, labels in train_loader:
        data = data.to(device)
        latent = encoder(data)
        latent_flat = latent.view(latent.size(0), -1)
        train_features.append(latent_flat.cpu())
        train_labels.append(labels)

train_features = torch.cat(train_features, dim=0).numpy()
train_labels = torch.cat(train_labels, dim=0).numpy()

# save extracted features
np.save("train_features.npy", train_features)
np.save("train_labels.npy", train_labels)

In [5]:
test_studio_dir = os.path.join(data_dir, 'test_studio')

test_studio_dataset = datasets.ImageFolder(root=test_studio_dir, transform=transform)
test_studio_loader = DataLoader(test_studio_dataset, batch_size=32, shuffle=False)

In [6]:
# repeat for test sets
test_studio_features = []
test_studio_labels = []

with torch.no_grad():
    for data, labels in test_studio_loader:
        data = data.to(device)
        latent = encoder(data)
        latent_flat = latent.view(latent.size(0), -1)
        test_studio_features.append(latent_flat.cpu())
        test_studio_labels.append(labels)

test_studio_features = torch.cat(test_studio_features, dim=0).numpy()
test_studio_labels = torch.cat(test_studio_labels, dim=0).numpy()

np.save("test_studio_features.npy", test_studio_features)
np.save("test_studio_labels.npy", test_studio_labels)

In [9]:
test_realworld_dir = os.path.join(data_dir, 'test_realworld')

test_realworld_dataset = datasets.ImageFolder(root=test_realworld_dir, transform=transform)
test_realworld_loader = DataLoader(test_realworld_dataset, batch_size=32, shuffle=False)

In [10]:
# repeat for test sets
test_realworld_features = []
test_realworld_labels = []

with torch.no_grad():
    for data, labels in test_realworld_loader:
        data = data.to(device)
        latent = encoder(data)
        latent_flat = latent.view(latent.size(0), -1)
        test_realworld_features.append(latent_flat.cpu())
        test_realworld_labels.append(labels)

test_realworld_features = torch.cat(test_realworld_features, dim=0).numpy()
test_realworld_labels = torch.cat(test_realworld_labels, dim=0).numpy()

np.save("test_realworld_features.npy", test_realworld_features)
np.save("test_realworld_labels.npy", test_realworld_labels)